In [ ]:
import arcpy as ap
import os
import cmath
from collections import defaultdict



# function to return coordinates from arc point object
def coordinates_of(point_object):
    x = round(point_object.X,5)
    y = round(point_object.Y,5)

    return x,y


def ten_meters_as_percentage(of_a_line):
    of_line_length = of_a_line.length
    percentage_of_line = float(10)/float(of_line_length)
    
    return percentage_of_line


# function to return coordinates of the point x meters from the start or end of a line:
def point_ten_meters_from_the_start(of_a_line):
    point_10m_from_start = of_a_line.positionAlongLine(ten_meters_as_percentage(of_a_line), True).firstPoint
    
    return coordinates_of(point_10m_from_start)
    
    
# function to return coordinates of the point x meters from the start or end of a line:
def point_ten_meters_from_the_end(of_a_line):
    point_10m_from_end = of_a_line.positionAlongLine(1 -ten_meters_as_percentage(of_a_line), True).firstPoint
    
    return coordinates_of(point_10m_from_end)

def hundred_meters_as_percentage(of_a_line):
    of_line_length = of_a_line.length
    percentage_of_line = float(100)/float(of_line_length)
    
    return percentage_of_line


# function to return coordinates of the point x meters from the start or end of a line:
def point_hundred_meters_from_the_start(of_a_line):
    point_100m_from_start = of_a_line.positionAlongLine(hundred_meters_as_percentage(of_a_line), True).firstPoint
    
    return coordinates_of(point_100m_from_start)
    
    
# function to return coordinates of the point x meters from the start or end of a line:
def point_hundred_meters_from_the_end(of_a_line):
    point_10m_from_end = of_a_line.positionAlongLine(1 - hundred_meters_as_percentage(of_a_line), True).firstPoint
    
    return coordinates_of(point_100m_from_end)


# function to calculate the angle between two sets of coordinatesdef angle_between(first_point_coordinates, last_point_coordinates):
def angle_between(point_a_coordinates, point_b_coordinates): 
    x1, y1 = float(point_a_coordinates[0]), float(point_a_coordinates[1])
    x2, y2 = float(point_b_coordinates[0]), float(point_b_coordinates[1])

    polarcoords = cmath.polar(complex(x2 - x1, y2 - y1))
    
    if math.degrees(polarcoords[1]) <= 90:
        degrees = 90 - math.degrees(polarcoords[1])
    else:
        degrees = 450 - math.degrees(polarcoords[1])

    return round(degrees, 1)


def angle_of_first_10m(of_a_line):
    the_lines = of_a_line
    first_point = coordinates_of(the_lines.firstPoint)
    last_point = coordinates_of(the_lines.lastPoint)
    point_10m_from_start = point_ten_meters_from_the_start(of_a_line)
    angle = angle_between(first_point, point_10m_from_start)
   
    return angle
    
    
def angle_of_last_10m(of_a_line):
    the_lines = of_a_line
    
    last_point = coordinates_of(the_lines.lastPoint)
    point_10m_from_end = point_ten_meters_from_the_end(of_a_line)
    angle = angle_between(last_point, point_10m_from_end)

    return angle

def angle_of_first_100m(of_a_line):
    the_lines = of_a_line
    first_point = coordinates_of(the_lines.firstPoint)
    last_point = coordinates_of(the_lines.lastPoint)
    point_10m_from_start = point_ten_meters_from_the_start(of_a_line)
    angle = angle_between(first_point, point_10m_from_start)
   
    return angle
    
    
def angle_of_last_100m(of_a_line):
    the_lines = of_a_line
    
    last_point = coordinates_of(the_lines.lastPoint)
    point_10m_from_end = point_ten_meters_from_the_end(of_a_line)
    angle = angle_between(last_point, point_10m_from_end)

    return angle


def angle_of_line_from_start(of_a_line):
    the_lines = of_a_line
    first_point = coordinates_of(the_lines.firstPoint)
    last_point = coordinates_of(the_lines.lastPoint)
    angle = angle_between(first_point, last_point)
    
    return angle 


def angle_of_line_from_end(of_a_line):
    the_lines = of_a_line
    first_point = coordinates_of(the_lines.firstPoint)
    last_point = coordinates_of(the_lines.lastPoint)
    angle = angle_between(last_point, first_point)
    
    return angle 


def list_to_string(list_of):
    list_as_string = str(list_of).replace('[', '').replace(']','').replace(' ','')
    
    return list_as_string


def GetDifference(b1, b2):
    r = (b2 - b1) % 360.0
    # Python modulus has same sign as divisor, which is positive here,
    # so no need to consider negative case
    if r >= 180.0:
        r -= 360.0
    return abs(r)

In [ ]:
local_shapefiles_directory = r'F:\Conflating_Networks\network_shapefiles'
local_geodatabase = r'F:\Conflating_Networks\working_files.gdb'

#local_shapefiles_directory = os.getcwd() + '\\network_shapefiles'
#local_geodatabase = os.getcwd() + '\\working_files.gdb'

ap.env.workspace = local_shapefiles_directory
network_shapefiles = ap.ListFeatureClasses()

ap.SpatialReference(102382)
spatial_reference = ap.SpatialReference(102382)

ap.env.workspace = local_geodatabase

In [ ]:
# creating necesarry files in local geodatabase
if ap.Exists(local_geodatabase) == False:
    ap.CreateFileGDB_management(os.getcwd(), 'working_files.gdb')
    
for each_network in network_shapefiles:
    network_name = ap.Describe(local_shapefiles_directory + '//' + each_network).baseName
    ap.env.workspace = local_geodatabase
    if ap.Exists('%s_to_line' % network_name) == False:       
        ap.FeatureToLine_management(local_shapefiles_directory + '//%s.shp' % network_name,
                                    '%s_to_line' % network_name)
        
    if ap.Exists('%s_dissolved_to_line' % network_name) == False:       
        ap.Dissolve_management(local_shapefiles_directory + '//%s.shp' % network_name, 
                                   'in_memory\\%s_dissolved'% network_name,
                                   '',
                                   '',
                                   'MULTI_PART',
                                   'DISSOLVE_LINES')
                
        ap.FeatureToLine_management('in_memory\\%s_dissolved' % network_name,
                                            '%s_dissolved_to_line' % network_name)
        
        ap.Delete_management('in_memory\\%s_dissolved' % network_name)

In [ ]:
for each_network in network_shapefiles:
    network_name = ap.Describe(local_shapefiles_directory + '\\' + each_network).baseName
  
    if ap.Exists('%s_crossings' % network_name) == False:
        ap.CreateFeatureclass_management(local_geodatabase, '%s_crossings' % network_name,
                                         'POINT',
                                         '',
                                         '',
                                         '',
                                         spatial_reference)
    else:
        ap.DeleteRows_management('%s_crossings' % network_name)            
        
    for field in ['intersections', 
                  'overpasses',
                  'dissolved_id',
                  'dissolved_angle_10m', 
                  'dissolved_angle_line', 
                  'notes']:

        
        if len(ap.ListFields('%s_crossings' % network_name)) < 10:
            ap.AddField_management('%s/%s_crossings' % (local_geodatabase, network_name),
                                   '%s_%s' % (network_name[:3], field),
                                   'STRING')

In [ ]:
overpass_dictionary = {}

for each_network in network_shapefiles:
    network_name = ap.Describe(local_shapefiles_directory + '\\' + each_network).baseName
    
    with ap.da.SearchCursor('%s_to_line' % network_name,
                        'FID_%s' % network_name,
                        sql_clause = (None,'ORDER BY FID_%s' % network_name))as overpass_cursor:
        
        overpass_dictionary[network_name] = []
        last_feature_id = None
    
        for each_feature in overpass_cursor:
            feature_id = each_feature[0]
        
            if feature_id == last_feature_id:
                overpass_dictionary[network_name].append(last_feature_id)
                
            else:
                pass
            
            last_feature_id = feature_id
    try:
        ap.MakeFeatureLayer_management('%s_to_line' % network_name,
                                   '%s_to_line_lyr' % network_name)
    except:
        pass

In [ ]:
for each_network in network_shapefiles:
    point_dictionary = defaultdict(list)
    network_name = ap.Describe(local_shapefiles_directory + '\\' + each_network).baseName

    with ap.da.SearchCursor('%s_dissolved_to_line' % network_name,
                        ['SHAPE@', 'OID@'],
                        '',
                        spatial_reference
                        ) as angle_cursor:
        
        for each_line in angle_cursor:
            the_lines = of_the_line = each_line[0]
            line_id = each_line[1]
            
            attrib_of_line_start = (angle_of_first_10m(of_the_line), angle_of_line_from_start(of_the_line), line_id)
            attrib_of_line_end = (angle_of_last_10m(of_the_line), angle_of_line_from_end(of_the_line), line_id)
            
            point_dictionary[coordinates_of(the_lines.firstPoint)].append(attrib_of_line_start)
            point_dictionary[coordinates_of(the_lines.lastPoint)].append(attrib_of_line_end)
 

    select_overpasses = '"FID_%s" IN %s' % (network_name, tuple(overpass_dictionary[network_name]))
    
    ap.SelectLayerByAttribute_management('%s_to_line_lyr' % network_name, 
                                         'NEW_SELECTION',
                                         select_overpasses)
    
    with ap.da.SearchCursor('%s_to_line_lyr' % network_name,
                        ['SHAPE@'],
                        '',
                        spatial_reference) as cursor_for_overpasses:
        
        for each_line in cursor_for_overpasses:
            the_lines = of_the_line = each_line[0]
         
            if coordinates_of(the_lines.firstPoint) in point_dictionary.keys():
                attrib_of_line_start = (angle_of_first_10m(of_the_line), angle_of_line_from_start(of_the_line))
                point_dictionary[coordinates_of(the_lines.firstPoint)].append(attrib_of_line_start) 
            
            if coordinates_of(the_lines.lastPoint) in point_dictionary.keys():
                attrib_of_line_end = (angle_of_last_10m(of_the_line), angle_of_line_from_end(of_the_line))
                point_dictionary[coordinates_of(the_lines.lastPoint)].append(attrib_of_line_end)
                
          
    with ap.da.InsertCursor('%s_crossings' % network_name, 
                           ['SHAPE@XY', 
                            '%s_dissolved_angle_10m' % network_name[:3],
                            '%s_dissolved_angle_line' % network_name[:3],
                            '%s_dissolved_id' % network_name[:3],
                            '%s_intersections' % network_name[:3],
                            '%s_overpasses' % network_name[:3]]) as cursor_for_points_with_angles:
        
        for each_point in point_dictionary.items():
            point = ap.CreateObject("Point")
            point.X, point.Y = float(each_point[0][0]), float(each_point[0][1])
            
            crossing_angles = []
            line_angles = []
            link_ids = []
            overpasses = []
            intersections = []
            
            for each_link in each_point[1]:        
                try:    
                    if len(each_link) == 3:
                        crossing_angles.append(each_link[0])
                        line_angles.append(each_link[1])
                        link_ids.append(each_link[2])
                    
                except:
                    overpasses.append(each_link[0])
            
            if crossing_angles != overpasses:
                try:
                    intersections = [x[0] for x in crossing_angles if x not in overpasses]
                except:
                    pass
                                                                         
            else:
                intersections = []
                
            cursor_for_points_with_angles.insertRow([point] + 
                                                    [list_to_string(crossing_angles),
                                                     list_to_string(line_angles),
                                                     list_to_string(link_ids),
                                                     list_to_string(sorted(intersections)), 
                                                     list_to_string(sorted(overpasses))])
    
    
        ap.AddXY_management('%s_crossings' % network_name)
    
    del cursor_for_points_with_angles
    del point_dictionary
    del overpass_dictionary[network_name]
    
    ap.Delete_management('%s_to_line_lyr' % network_name)

del overpass_dictionary

In [ ]:
if ap.Exists("DTD_crossings_near_Nav_crossings") == False:
    
    ap.GenerateNearTable_analysis(in_features="DTD_2010_Links_directions_crossings", 
                                 near_features="NavStreets_CDOT_TMC14Q1_WM84_crossings", 
                                 out_table='DTD_crossings_near_Nav_crossings', 
                                 search_radius="50 Meters", 
                                 location="NO_LOCATION", 
                                 angle="ANGLE", 
                                 closest="ALL", 
                                 closest_count="10", 
                                 method="GEODESIC")

In [ ]:
ap.JoinField_management('DTD_crossings_near_Nav_crossings', 
                        "IN_FID", 
                        "DTD_2010_Links_directions_crossings", 
                        "OBJECTID", 
                        '''DTD_crossings;DTD_crossings_link_id;
                        DTD_intersections;
                        DTD_overpasses;
                        DTD_dissolved_angle_10m;
                        DTD_dissolved_angle_line;
                        DTD_dissolved_id;
                        POINT_X;
                        POINT_Y''')


ap.JoinField_management('DTD_crossings_near_Nav_crossings',
                           "NEAR_FID", 
                           "NavStreets_CDOT_TMC14Q1_WM84_crossings",
                           "OBJECTID", 
                           '''Nav_crossings;
                           Nav_crossings_link_id;
                           Nav_intersections;
                           Nav_overpasses;
                           Nav_dissolved_angle_10m;
                           Nav_dissolved_angle_line;
                           Nav_dissolved_id;
                           POINT_X;
                           POINT_Y''')

for field in ['matched_Nav_angle_to_matched_DTD_angle_line', 
               'matched_Nav_id_to_matched_DTD_id_line',  
               'matched_Nav_angle_to_matched_DTD_angle_10m', 
               'matched_Nav_id_to_matched_DTD_id_10m', 
               'matched_notes']:
    try:
        ap.AddField_management('DTD_crossings_near_Nav_crossings', 
                          field, 
                          'TEXT')
    except:
        pass

for each_network in network_shapefiles:
    network_name = ap.Describe(local_shapefiles_directory + '\\' + each_network).baseName
    try:
        ap.Delete_management('%s_crossings_lyr' % network_name)
                                   
    except:
        pass

if ap.Exists('DTD_crossings_near_Nav_crossings_with_matches') == False:
    ap.TableToTable_conversion ('DTD_crossings_near_Nav_crossings', 
                            local_geodatabase, 
                            'DTD_crossings_near_Nav_crossings_with_matches')

ap.Delete_management('DTD_crossings_near_Nav_crossings')

In [ ]:
def GetDifference(b1, b2):
    r = (b2 - b1) % 360.0
    if r >= 180.0:
        r -= 360.0
    return abs(r)

with ap.da.UpdateCursor('DTD_crossings_near_Nav_crossings_with_matches', 
                        ['Nav_dissolved_angle_10m',
                         'DTD_dissolved_angle_10m',
                         'Nav_dissolved_angle_line',
                         'DTD_dissolved_angle_line',
                         'Nav_dissolved_id',
                         'DTD_dissolved_id',
                         'matched_Nav_angle_to_matched_DTD_angle_line', 
                         'matched_Nav_id_to_matched_DTD_id_line',  
                         'matched_Nav_angle_to_matched_DTD_angle_10m', 
                         'matched_Nav_id_to_matched_DTD_id_10m']) as cursor_to_match_crossings:

    for matches in cursor_to_match_crossings:
        nav_10m_angles = list_to_string(matches[0])
        nav_line_angles = list_to_string(matches[2])
        nav_ids = list_to_string(matches[4])
        
        dtd_10m_angles = list_to_string(matches[1])
        dtd_line_angles = list_to_string(matches[3])
        dtd_ids = list_to_string(matches[5])
        
        matched_10m_ids = []
        matched_10m_angles = []
        
        for nav_angle in nav_10m_angles.split(','):
            
            try:
                floating_nav_angle = float(nav_angle.replace(' ',''))   
            
            except:
                continue
            
            for dtd_angle in dtd_10m_angles.split(','):
                
                try:
                    floating_dtd_angle = float(dtd_angle)

                except:
                    continue
                
                if GetDifference(floating_nav_angle, floating_dtd_angle) < 6:
                    nav_id = int(nav_ids.split(',')[nav_10m_angles.split(',').index(nav_angle)])
                    dtd_id = int(dtd_ids.split(',')[dtd_10m_angles.split(',').index(dtd_angle)])
                    matched_10m_angles.append((floating_nav_angle, floating_dtd_angle))
                    matched_10m_ids.append((nav_id,dtd_id))
                    
        matched_line_ids = []
        matched_line_angles = []
        
        for nav_angle in nav_line_angles.split(','):
            
            try:
                floating_nav_angle = float(nav_angle.replace(' ',''))
                
            except:
                continue
            
            for dtd_angle in dtd_line_angles.split(','):
                
                try:
                    floating_dtd_angle = float(dtd_angle)

                except:
                    continue
                
                if GetDifference(floating_nav_angle, floating_dtd_angle) < 6:
                    dtd_id = int(dtd_ids.split(',')[dtd_line_angles.split(',').index(dtd_angle)])
                    nav_id = int(nav_ids.split(',')[nav_line_angles.split(',').index(nav_angle)])
                    matched_line_angles.append((floating_nav_angle, floating_dtd_angle))
                    matched_line_ids.append((nav_id,dtd_id))
        
        matches[6] = list_to_string(matched_line_angles)
        matches[7] = list_to_string(matched_line_ids)
        matches[8] = list_to_string(matched_10m_angles)
        matches[9] = list_to_string(matched_10m_ids)
        
        cursor_to_match_crossings.updateRow(matches)

del cursor_to_match_crossings

In [ ]:
with ap.da.UpdateCursor('DTD_crossings_near_Nav_crossings_with_matches', 
                        ['Nav_dissolved_id',
                         'DTD_dissolved_id',  
                         'matched_Nav_id_to_matched_DTD_id_10m',
                         'matched_notes']) as cursor_for_10m_matches:
    

    for exact_match in cursor_for_10m_matches:
        try:
            all_nav_ids = [int(x) for x in exact_match[0].split(',')]
            all_dtd_ids = [int(x) for x in exact_match[1].split(',')]
        
        except:
            continue
    
        try:
            matched_ids = exact_match[2].split('),(')
            matched_nav_ids = [int(x.split(',')[0].replace('(','')) for x in exact_match[2].split('),(')]
            matched_dtd_ids = [int(x.split(',')[1].replace(')','')) for x in exact_match[2].split('),(')]
            
            if all_nav_ids == matched_nav_ids:
                if all_dtd_ids == matched_dtd_ids:
                    exact_match[3] = 'all ids matched at 10m along line'
                else:
                    pass
            else:
                pass
        
        except:
            pass
        
        cursor_for_10m_matches.updateRow(exact_match)   

In [ ]:
with ap.da.UpdateCursor('DTD_crossings_near_Nav_crossings_with_matches', 
                        ['Nav_dissolved_id',
                         'DTD_dissolved_id',  
                         'matched_Nav_id_to_matched_DTD_id_line',
                         'matched_notes']) as cursor_for_10m_matches:
    

    for exact_match in cursor_for_10m_matches:
        try:
            all_nav_ids = [int(x) for x in exact_match[0].split(',')]
            all_dtd_ids = [int(x) for x in exact_match[1].split(',')]
        
        except:
            continue
    
        try:
            matched_ids = exact_match[2].split('),(')
            matched_nav_ids = [int(x.split(',')[0].replace('(','')) for x in exact_match[2].split('),(')]
            matched_dtd_ids = [int(x.split(',')[1].replace(')','')) for x in exact_match[2].split('),(')]
            
            if all_nav_ids == matched_nav_ids:
                if all_dtd_ids == matched_dtd_ids:
                    exact_match[3] = exact_match[3] +', all ids matched at 10m along length of line'
                else:
                    pass
            else:
                pass
        
        except:
            pass
        
        cursor_for_10m_matches.updateRow(exact_match)   

In [ ]:
matches = []
with ap.da.SearchCursor("DTD_crossings_near_Nav_crossings_with_matches",
                        ['OID@','POINT_X', 'POINT_Y', 'POINT_X_1', 'POINT_Y_1']) as matched_point_cursor:
    
    for row in matched_point_cursor:
        matches.append(row)

if ap.Exists('matched_lines') == False:
    ap.CreateFeatureclass_management(local_geodatabase, 'match_lines',
                                         'POLYLINE',
                                         '',
                                         '',
                                         '',
                                         spatial_reference)

with ap.da.InsertCursor('match_lines', 
                           ['OID@','SHAPE@XY']) as cursor_for_matching_lines:
        
        for each_line in matches:
            
            if None not in each_line:
                match_id = each_line[0]
                
                point_dtd= ap.CreateObject("Point")
                point_dtd.X, point_dtd.Y = float(each_line[1]), float(each_line[2])

                point_nav= ap.CreateObject("Point")
                point_nav.X, point_nav.Y = float(each_line[1]), float(each_line[2])
                
                line = ap.Polyline(point_dtd, point_nav)
            
                cursor_for_matching_lines.insertRow([match_id,line])
            
            else:
                pass